In [4]:
# env setting
import sys
sys.path.append("../src")
sys.path.append("../models")

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
# from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer

# model import
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

#custom function
import config
import helpers
from custom_pipeline import Custom_Pipeline

In [59]:
from numpy import ndarray
import pandas as pd
import numpy as np
from sklearn._typing import ArrayLike, MatrixLike

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

def feature_transform(X):
    train_bool = X.columns.isin(["outcome"]).sum()
    """ Add New Features
    """
    # Not works but drop score when removing
    # X['treated_more_than_once'] = X['hospital_number'].apply(
    #     lambda x: 1 if X['hospital_number'].value_counts()[x] > 1 else 0
    # )
    
    X['number_of_treatements'] = X['hospital_number'].apply(
        lambda x: X['hospital_number'].value_counts()[x]
    )
    
    X['deviation_from_normal_temp'] = X['rectal_temp'].apply(
        lambda x: abs(x - 37.8)
    )
    
    # feature importane 0 but drop score when removing
    # X['lesion_2'] = X['lesion_2'].apply(
    #     lambda x: 2 if '11' in str(x) else (0 if str(x) == '0' else 1)
    # )
    
    X['surgery'] = X['surgery'].apply(
        lambda x: 1 if x == 'yes' else 0
    )
    
    if train_bool:
        X['pain'] = X['pain'].replace({
            'slight' : 'mild_pain'
        })
    
    else:
        X['pain'] = X['pain'].replace({
            'moderate' : 'mild_pain'
        })
        
    X['capillary_refill_time'] = X['capillary_refill_time'].replace({
        '3' : 'more_3_sec'
    })
    
    X['peristalsis'] = X['peristalsis'].replace({
        'distend_small' : np.nan
    })
    
    X['nasogastric_reflux'] = X['nasogastric_reflux'].replace({
        'slight' : np.nan
    })
    
    X['rectal_exam_feces'] = X['rectal_exam_feces'].replace({
        'serosanguious' : np.nan
    })

    return X
    

class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self._train_count = 0
        self._global_mean = 0
        self._n_rows = 0
        self._smoothing_memory = {}
        self.features = features
        
    def fit(self, X, y, alpha=0.5):
        self.X = X
        
        self.encoded_cols = []
        if len(self._smoothing_memory) == 0:
            self._n_rows = self.X.shape[0]
            self.X['outcome'] = y
            self._global_mean = self.X['outcome'].mean()
            
            for col in self.features:
                target_mean = self.X.groupby(col)['outcome'].mean()
                smoothing = (target_mean * self._n_rows + self._global_mean * alpha) / (self._n_rows + alpha)
                self._smoothing_memory[col] = smoothing
                
                # transform
                encoded_col = self.X[col].map(smoothing)
                self.encoded_cols.append(pd.DataFrame({col:encoded_col}))
                
        else:
            for col in self.features:
                smoothing = self._smoothing_memory[col]
                encoded_col = self.X[col].map(smoothing)
            
                self.encoded_cols.append(pd.DataFrame({col:encoded_col}))
        
        return self
        
    def transform(self, X, y = None):
        self.X.drop(['outcome'], axis=1)
        
        results = pd.concat(self.encoded_cols, axis=1)
        self.X = pd.concat([self.X.drop(self._features, axis=1), results], axis=1)
        
        return self.X
    
    def fit_transform(self, X, y, alpha=0.5):
        return self.fit(X, y).transform(X)

ModuleNotFoundError: No module named 'sklearn._typing'

In [60]:
(train, test, origin, submission) = helpers.data_loader()
train.head(5)

train = train.replace('None', np.NaN)
test = test.replace('None', np.NaN)
origin = origin.replace('None', np.NaN)

categorical_features = config.CATEGORICAL_FEATURES
categorical_features.append('number_of_treatements')
# categorical_features.remove('lesion_2')
# categorical_features.remove('lesion_3')
target = 'outcome'

numerical_features = list(set(train.columns) - set(categorical_features) - set(config.USELESS_FEATURES) - set([target]))

X = pd.concat(
    [train, origin], ignore_index=True
)
X = train.drop_duplicates()

print(X.shape)
print(test.shape)

y = X.pop('outcome')

(1235, 29)
(824, 28)


In [61]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

categorical_features = ['cp_data']

y_map = {k : v for v, k in enumerate(train.outcome.unique())}
y_reverse_map = {v : k for k, v in y_map.items()}
y = y.map(y_map)

pipe = Pipeline(steps=[
    ('preprocessor', TargetEncoder(categorical_features))
])


In [62]:
pipe.fit(X, y)

AttributeError: 'TargetEncoder' object has no attribute 'features'

AttributeError: 'TargetEncoder' object has no attribute 'features'

AttributeError: 'TargetEncoder' object has no attribute 'features'